## Introduction

After segments have been created for a given image, it is pivotal to characterize them in a meaningful manner. The power of OBIA partly relies on the fact that a variety of descriptors can be used to characterize segments beyond their spectral properties. Quite often shape, textural or contextual features are beneficial for a classification of the segments. The following notebook serves to explore some basic possibilities to calculate such features.

Note that term "features" as it used currently, is somewhat ambiguous or imprecise. From the point of the computer vision domain, what we are actually interested in are rather descriptors than features. To be precise: Features are commonly understood to be properties/parts of the image that are of particular interest (-> edges, corners, lines, keypoints, etc), whereas descriptors are used to characterize the features or the image as a whole (-> local vs. global descriptors). However, in the context of machine learning features are input variables used to characterize samples and since the descriptors are often serving this purpose, the exact differentiation of both terms will not be considered further in the following.

When it comes to calculating features in python, a way needs to be found to aggregate the information of all pixels for a given segment according to the definition of the specific feature of interest. Depending on the chosen framework, there are different ways to do so. Using the data cube friendly (rio-)xarray package, one can rely on pandas-like groupby operations, for example. Usually somewhat more performant is the `regionprops()` function from [skimage](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops_table). It renders the need for manual grouping obsolete and comes along with some already pre-implemented features. Therefore, we will mainly focus on this one below.

## Setup

In [ ]:
# to run this on google colab
if 'google.colab' in str(get_ipython()):
    import os
    repo_dir = "obia_tutorials"
    marker_file = os.path.join(repo_dir, ".setup_done")    
    if not os.path.exists(marker_file):
        !git clone https://github.com/fkroeber/obia_tutorials.git
        !pip install -r obia_tutorials/requirements.txt
        with open(marker_file, 'w') as f:
            f.write("Setup completed")
    if not os.getcwd().endswith(repo_dir):
        os.chdir(repo_dir)

In [ ]:
# imports
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import mahotas as mh
import networkx as nx
import numpy as np
import pandas as pd
import plotly.express as px
import rioxarray as rxr
import xarray as xr
import time

from matplotlib import rcParams
from scipy import stats
from skimage import graph
from skimage.color import label2rgb
from skimage.measure import regionprops, regionprops_table
from skimage.morphology import binary_dilation
from skimage.segmentation import mark_boundaries, slic
from skimage.util import map_array
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from tqdm import tqdm

# set general figsize
rcParams['figure.figsize'] = (7.5, 7.5)

## Exemplary data

For demonstration purposes, we will use a 4-band R,G,B,NIR image segmented with SLIC. The segmentation parameters below were chosen by trial-and-error until objects of interest (e.g. trees and their shadows) appeared to be segmented in a meaningful manner.

In [ ]:
# read image
img_path = "sample_data/ortho_subset_I.tif"
ds = xr.open_dataset(img_path)

# extract arrays
arr = np.transpose(np.array(ds.to_array()[0]), (1,2,0))
rgb = arr.astype(np.uint8)[...,[0,1,2]]
ndvi = (arr[...,3] - arr[...,0])/(arr[...,3] + arr[...,0])

# perform segmentation
segments = slic(arr, n_segments=3000, compactness=0.3)

**<span style="color:orange">Task 1: Print the shape of the image and the number of segments.</span>**

In [ ]:
# code goes here

In [ ]:
# visualise rgb & segmentation
fig, axs = plt.subplots(ncols=3, figsize=(15,5), constrained_layout=True)
axs[0].imshow(rgb)
axs[1].imshow(mark_boundaries(rgb, segments, (1,0,0), mode="thick"))
axs[2].imshow(label2rgb(segments, rgb, kind='avg').astype(int))

axs[0].set_title("original rgb")
axs[1].set_title("segmentation boundaries")
axs[2].set_title("mean rgb per segment")

for ax in axs:
    ax.set_axis_off();

Note that this areal image is a very high-resolution image and quite large (1500 x 1500 pxls). This allows to investigate the scalability of the calculation of the proposed feature descriptors. To this end, a decorator for timing the execution of some subsequently defined functions is defined. If you want to get the timing more accurately, you can put the magic `%%timeit` at the beginning of a chuck to repeat the cell execution multiple times and get an average. For a rough estimate, however, the function decorator below should be sufficient.

In [ ]:
# decorator for timing
def timeit(method):
    def timed(*args, **kw):
        tstart = time.perf_counter()
        result = method(*args, **kw)
        tend = time.perf_counter()
        total_ms = (tend - tstart)
        print(f"Execution time: {total_ms:.2f}s")
        return result   
    return timed

## Types of feature descriptors

### I. Spectral features

Very intuitive and fast to calculate are the measures of central tendency and variation, the most popular of which are the mean spectral value for a segment and its standard deviation. The calculation of the mean value is pretty straightforward and was already carried out implicitly by using the function `label2rgb` above. For non-visualisation related purposes, the pre-implemented mean calculation provided by `regionprops` can be used as shown below.

In [ ]:
# define mean calculation function
@timeit
def calc_mean(seg_arr, img_arr):
    spec_feats = regionprops_table(
        label_image = seg_arr, 
        intensity_image = img_arr,
        properties = ["label", "intensity_mean"]
        )
    return pd.DataFrame(spec_feats)  

# perform calculation & get timing
means = calc_mean(segments, np.dstack([rgb, ndvi]))
display(means)

For the calculation of the standard deviation we can write a custom extra_property function to be passed to regionprops. These functions need to take a regionmask and (optionally) an intensity mask as input and define a function that transforms these 2D array into an output value for a given segment. It is important to note that for many intensity-based calculations, the regionmask (which is a binary array) needs to be used to first filter all intensity values that actually belong to the segment for the given intensity mask. Otherwise, the calculation is performed on the intensity array that represents the bounding box rather than the mask of the segment. The example for the standard deviation demonstrates the basic usage of the extra_properties.   

**<span style="color:orange">Task 2: Implement the calculation of the standard deviation as an extra_property below.</span>**

In [ ]:
# define stdv function
def std(regionmask, intensity_img):
    # code goes here 
    return std
    
@timeit
def calc_std(seg_arr, img_arr):
    spec_feats = regionprops_table(
        label_image = seg_arr, 
        intensity_image = img_arr,
        properties = ["label",],
        extra_properties=(std,)
        )
    return pd.DataFrame(spec_feats)  

# perform calculation & get timing
stds = calc_std(segments, np.dstack([rgb, ndvi]))
display(stds)

Note that at the moment, regionprops doesn't support reducing the information across bands. Instead, all the intensity-specific functions are applied channel-wise. Since band-wise information is often desired, this is not too much of a problem. In the example above, calculating the std across r,g,b bands and nir wouldn't be very reasonable due to the different scale of the single variables. For the sake of completeness, however, the following shows one way of calculating the standard deviation across bands (using only rgb). For this purpose, a transformation to xarrays followed by grouping according to the segmentation layer is used. 

In [ ]:
# define stdv across bands function
@timeit
def calc_std_3d(seg_arr, img_arr):
    seg_arr = xr.DataArray(seg_arr)
    img_arr = xr.DataArray(img_arr)
    stds = img_arr.groupby(seg_arr).std(dim=...)
    return pd.DataFrame({"label": stds.group, "std": np.array(stds)})

# perform calculation & get timing
stds_3d = calc_std_3d(segments, rgb)
display(stds_3d)

To visualize the results there is a helper function called [skimage.util.map_array](https://scikit-image.org/docs/stable/api/skimage.util.html#skimage.util.map_array). 

**<span style="color:orange">Task 3: Take a look at the documentation of the function map_array to visualise the results that we've calculated above.</span>**

In [ ]:
# mapping NDVI vals back to image segments

# code goes here
# mapped_mean = ...
# mapped_std = ...

# visualise results
fig, axs = plt.subplots(ncols=4, figsize=(20,10), constrained_layout=True)
axs[0].imshow(mark_boundaries(rgb, segments, (1,0,0), mode="thick"))
axs[1].imshow(ndvi, cmap="YlGn")
axs[2].imshow(mapped_mean, cmap="YlGn")
axs[3].imshow(mapped_std)

axs[0].set_title("rgb & boundaries")
axs[1].set_title("ndvi per pixel")
axs[2].set_title("mean ndvi per segment")
axs[3].set_title("std ndvi per segment")

for ax in axs.flat:
    ax.set_axis_off();

Mapping the obtained values back to the segments array shows that - as expected - pure road segments have very low NDVI values and variations. However, segments that include several objects, such as cars parked near trees, stand out as inhomogeneous. Vegetation segments have strong average NDVI values and a medium to high variability. The water surface appears as a very homogenous region with hardly any differences in mean and variation of NDVI among segments belonging to this group. 

### II. Shape features

Starting with some basic shape features, one may want to calculate the area of the bounding box of segments to get an indication for their extent. Given the bounding box one can further calculate features such as rectangularity defined as the ratio between a segment's area and its bbox.        

**<span style="color:orange">Task 4: Implement the calculation of the rectangularity.</span>**

In [ ]:
# define rectangularity
def rectangularity(regionmask):
    # code goes here
    
@timeit
def calc_rect_shapes(seg_arr):
    shp_feats = regionprops_table(
        label_image = seg_arr, 
        properties = ["label", "area_bbox"],
        extra_properties=(rectangularity,)
        )
    return pd.DataFrame(shp_feats)  

# perform calculation & get timing
rects = calc_rect_shapes(segments)
display(rects)

Note that rectangularity as defined above isn't rotation-invariant, i.e. it value changes depending on the orientation of a segment. Since many natural objects can have various orientations while still being the same object (e.g. parked cars), it is therefore rarely a meaningful feature. 

Conceptually stronger are features such as compactness or circularity, which relates the area of an object to its perimeter (called shape-index in eCognition). Another sometimes useful feature is solidity defined as the ratio of pixels in a segment to pixels of its convex hull. The later one is also pre-implemented in regionprops.

In [ ]:
# define compactness
@timeit
def calc_shapes(seg_arr, solidity=True):
    if solidity:
        props = ["label", "solidity", "area", "perimeter"]
    else:
        props = ["label", "area", "perimeter"]
    shp_feats = regionprops_table(
        label_image = seg_arr, 
        properties = props,
        )
    compactness = 4*np.pi*shp_feats["area"]/(shp_feats["perimeter"]**2)
    shp_feats["compactness"] = compactness
    shp_feats.pop("area")
    shp_feats.pop("perimeter")
    return pd.DataFrame(shp_feats)  

# perform calculation & get timing
shapes = calc_shapes(segments)
display(shapes)

In [ ]:
# mapping vals back to image segments
mapped_compactness = map_array(segments, np.array(shapes["label"]), np.array(shapes["compactness"]))
mapped_solidity = map_array(segments, np.array(shapes["label"]), np.array(shapes["solidity"]))

# visualise stdvs
fig, axs = plt.subplots(ncols=3, figsize=(15,10), constrained_layout=True)
axs[0].imshow(mark_boundaries(rgb, segments, (1,0,0), mode="thick"))
axs[1].imshow(mapped_compactness)
axs[2].imshow(mapped_solidity)

axs[0].set_title("rgb & boundaries")
axs[1].set_title("compactness")
axs[2].set_title("solidity")

for ax in axs.flat:
    ax.set_axis_off();

**<span style="color:orange">Task 5: Which feature descriptor (compactness vs. solidity) seems favourable to you? Do they differ in terms of computational costs? Run the calculation with and without solidity to evaluate the difference.</span>**

<i>Answer goes here</i>

In [ ]:
# code goes here

Talking about scalability, the family of the so-called [image moments](https://en.wikipedia.org/wiki/Image_moment) should be mentioned as an additional set of computationally favorable descriptors. As weighted averages of the image pixels' intensities they provide information about the orientation and symmetry of a segment. One important formulation of image moments are the Hu moments. They extend the basic central moment definitions in a way that the resulting set of features is invariant to certain transformations. These transformations include translation (movement), rotation, and scaling (size changes), which makes them powerful feature descriptors.

**<span style="color:orange">Task 6: Insert the corresponding property for the Hu moments below.</span>**

In [ ]:
@timeit
def calc_moments(seg_arr):
    shp_feats = regionprops_table(
        label_image = seg_arr, 
        # code goes here
        # properties = ["label", ...],
        )
    return pd.DataFrame(shp_feats)  

# perform calculation & get timing
moments = calc_moments(segments)
display(moments)

In [ ]:
# visualise hu moments
fig, axs = plt.subplots(ncols=4, nrows=2, figsize=(20,10), constrained_layout=True)

# mapping vals back to image segments
for i, ax in enumerate(axs.flat):
    if i == 0:
        ax.imshow(mark_boundaries(rgb, segments, (1,0,0), mode="thick"))
        ax.set_title("rgb & boundaries")
    else:
        mapped_moments = map_array(segments, np.array(moments["label"]), np.array(moments.iloc[:,i]))
        ax.imshow(mapped_moments)
        ax.set_title(moments.iloc[:,i].name)

for ax in axs.flat:
    ax.set_axis_off();

The interpretation of the single moments is somewhat difficult due to their missing direct correspondence to physical aspects of the shapes. They are rather constructed in a mathematical way to satisfy the above-mentioned invariance properties. One can generalize that the 1st Hu moment (moments_hu_0) represents the overall symmetry of the object around its centre of mass. It's sensitive to elongation, meaning that elongated shapes will have a higher value for this moment. This sensitivity to elongation is also inherent to the 2nd Hu moment which is related to the object's symmetry along the major and minor axes (note that the shape descriptors `axis_major_length` and `axis_minor_length` provided by skimage are therefore based on image moment calculation under the hood). Higher order moments often relate to other aspects of the segments' shapes such as skewness and kurtosis. As we are dealing with a superpixel segmentation, the shapes are rather symmetrical and therefore less prominent in the corresponding visualizations.  

### III. Textural features

Textural features are commonly calculated globally, i.e. for the whole image. However, there is nothing that prevents us from calculating them for each segment individually. Famous texture descriptors are the Haralick features which are based on the 2D histogram distributions of the intensity differences between neighboring pixels. These so-called grey-level co-occurrence matrices (GLCM) can be used to derive a set of descriptors such as the angular second moment, contrast, correlation,...

Computing the Haralick features locally for segments requires some pre-processing by buffering the segments to avoid errors in case of thin segments that don't have any neighbor in a certain direction. Also note that the Haralick features are based on single band greyscale images and require them to be non-negative integers - if we want to calculate them for the NDVI, we thus have to rescale the values first.

In [ ]:
@timeit
def calc_haralick_feats(seg_arr, img_arr):
    """
    Calculates the 13 stable Haralick features for image segments,
    based on mahotas as more efficient & comprehensive implementation compared to skimage
    """
    seg_arr = seg_arr.astype(np.uint16)
    img_arr = img_arr.astype(np.uint8)
    regions = regionprops(seg_arr)
    region_props = {}
    for region in tqdm(regions):
        # create dilated region mask to avoid boundary effects
        mask = seg_arr == region.label
        mask = binary_dilation(mask)
        # clip image to bbox
        [rows, cols] = np.where(mask)
        row1, row2 = min(rows), max(rows)+1
        col1, col2 = min(cols), max(cols)+1
        mask_clipped = mask[row1:row2, col1:col2]
        img_clipped = img_arr[row1:row2, col1:col2]
        # fill background with zeros
        clip = np.where(mask_clipped, img_clipped, 0)
        # calculate texture features for different orientations & average
        feats = mh.features.haralick(clip, ignore_zeros=True).mean(0)
        region_props[region.label] = feats
    # summarise results
    haralick_df = pd.DataFrame(region_props).T
    haralick_cols = [f"haralick-{x}" for x in np.arange(haralick_df.shape[1])] 
    haralick_df.columns = haralick_cols
    return haralick_df

# perform calculation & get timing
haralicks = calc_haralick_feats(segments, (100*(ndvi+1)).astype(np.uint8))
display(haralicks)

**<span style="color:orange">Task 7: Why did this calculation take so long? Can you think of other textural descriptors that may be computationally cheaper?</span>**

<i>Answer goes here</i>

Let us take a look at some of the results and see if it was worth it to perform this expensive calculation.

In [ ]:
# visualise results
mapped_asm = map_array(segments, np.unique(segments), np.array(haralicks["haralick-0"]))
mapped_contrast = map_array(segments, np.unique(segments), np.array(haralicks["haralick-1"]))
mapped_correlation = map_array(segments, np.unique(segments), np.array(haralicks["haralick-2"]))

fig, axs = plt.subplots(ncols=4, figsize=(20,10), constrained_layout=True)
axs[0].imshow(mark_boundaries(rgb, segments, (1,0,0), mode="thick"))
axs[1].imshow(mapped_asm)
axs[2].imshow(mapped_contrast)
axs[3].imshow(mapped_correlation)

axs[0].set_title("rgb & boundaries")
axs[1].set_title("angular second moment")
axs[2].set_title("contrast")
axs[3].set_title("correlation")

for ax in axs.flat:
    ax.set_axis_off();

**<span style="color:orange">Task 8: Describe and interpret the results briefly. Do these Haralick features over some additional information compared to the previously calculated ones?</span>**

<i>Answer goes here</i>


If we want to find a cheaper substitution for this textural descriptor, we may look for first-order metrics. They operate on the greyscale values and their distribution directly rather than analyzing similarities between pixel pairs. One of these first-order metrics, the variance, was already introduced in the section of spectral features. It inherently conveys information on the magnitude of greyscale differences for a given segment. Another popular distribution-based metric for the degree of disorder of values is the entropy measure as shown subsequently. Note that the entropy measure is based on the bins of the greyscale histogram and therefore either requires a common scaling of input bands or a binning schema that fits the specific band of interest.  

In [ ]:
def entropy_ndvi(regionmask, intensity_img):
    vals = intensity_img[regionmask]
    arr = stats.relfreq(vals, 100, defaultreallimits=(-1,1))[0]
    return stats.entropy(arr)

@timeit
def calc_entropy(seg_arr, img_arr):
    entropy_feats = regionprops_table(
        label_image = seg_arr, 
        intensity_image = img_arr, 
        properties = ["label"],
        extra_properties = (entropy_ndvi,)
        )
    return pd.DataFrame(entropy_feats)  

# perform calculation & get timing
entropy = calc_entropy(segments, ndvi)
display(entropy)

In [ ]:
# visualise results
mapped_entropy = map_array(segments, np.unique(segments), np.array(entropy["entropy_ndvi"]))

fig, axs = plt.subplots(ncols=2, figsize=(7.5,7.5), constrained_layout=True)
axs[0].imshow(mark_boundaries(rgb, segments, (1,0,0), mode="thick"))
axs[1].imshow(mapped_entropy)

axs[0].set_title("rgb & boundaries")
axs[1].set_title("entropy")

for ax in axs.flat:
    ax.set_axis_off();

### IV. Contextural features

Contextual features concern the neighborhood of a given segment and are particularly interesting so they allow the characterization of segments with a larger effective field of view. To calculate such features the representation of a segmentation as a so-called Region Adjacency Graph (RAG) can be leveraged. In this graph-based representation, the segments are represented by nodes connected to their neighbors via edges. Skimage provides RAG functionality based on the underlying Networkx library.  

To demonstrate the usage of contextual features we will focus on the spectral value of the neighbors. However, the following examples can easily be extended to other features of the neighboring segments (such as shape features).  

In [ ]:
@timeit
def calc_mean_std_neighbours(seg_arr, img_arr):
    # calculate region adjacency graph
    rag = graph.RAG(seg_arr)
    # get properties of neighbours that are of interest
    neighbor_attrs = regionprops_table(
        label_image = seg_arr, 
        intensity_image = img_arr,
        properties = ["label", "mean_intensity"],
        )
    neighbor_attrs = pd.DataFrame(neighbor_attrs)
    neighbor_attrs.set_index("label", inplace=True)
    # accumulate data of neighbouring segments
    neighbor_data = []
    for node, neighbors in rag.adjacency():
        nn_attrs = neighbor_attrs.loc[list(map(int, neighbors))]
        nn_means = nn_attrs.mean()
        nn_std = nn_attrs.std()
        neighbor_data.append([node, len(neighbors), *nn_means, *nn_std])
    # compile results
    neighbor_attrs_df = pd.DataFrame(neighbor_data).set_index(0)
    neighbor_feats = neighbor_attrs_df.fillna(0)
    neighbor_feats.index.names = ['label']
    n_bands_img_arr = img_arr.shape[-1] if len(img_arr.shape) == 3 else 1
    neighbor_feats.columns = [
        'neighbours_n',
        *[f'neighbours_mean_{i}' for i in np.arange(n_bands_img_arr)],
        *[f'neighbours_std_{i}' for i in np.arange(n_bands_img_arr)],
        ]
    return neighbor_feats.reset_index()

# perform calculation & get timing
neighb_vals = calc_mean_std_neighbours(segments, rgb)
display(neighb_vals)

The real power of a graph-based representation, however, lies in the beauty of graph-based mathematics. They allow more efficient calculations, which render the need of the nasty for-loop used above. In particular, a 2D adjacency matrix describing the neighboring relationship between nodes can be used to calculate the information on neighbors means and stdvs in a faster way by using numpy dot product calculations. Note that the result is the same as above.  

In [ ]:
@timeit
def calc_mean_std_neighbours(seg_arr, img_arr):
    # create region adjacency graph & matrix
    rag = graph.RAG(seg_arr)
    adj_matrix = nx.adjacency_matrix(rag)
    # get properties of neighbours that are of interest
    neighbor_attrs = regionprops_table(
        label_image = seg_arr, 
        intensity_image = img_arr,
        properties = ["label", "mean_intensity"],
        )
    neighbor_attrs = pd.DataFrame(neighbor_attrs)
    neighbor_attrs.set_index("label", inplace=True)
    # get number of neighbours
    num_neighbors = adj_matrix.sum(axis=1) 
    # convert neighbor_attrs to a numpy array
    neighbor_attrs_array = neighbor_attrs.reindex(list(rag.nodes)).to_numpy()
    # get mean vals
    neighbor_sum = adj_matrix.dot(neighbor_attrs_array)
    neighbor_mean = neighbor_sum/(num_neighbors.reshape(-1,1))
    # get std vals
    neighbor_attrs_sq_sum = adj_matrix.dot(np.square(neighbor_attrs_array))
    num_neighbors_bessel = np.maximum(num_neighbors - 1, 1)[:, None]
    neighbor_attrs_sq_sum_bessel = np.divide(neighbor_attrs_sq_sum, num_neighbors_bessel, where=num_neighbors_bessel != 0)
    neighbor_var = (
        neighbor_attrs_sq_sum_bessel - 
        np.divide(np.square(neighbor_sum), num_neighbors[:, None] * num_neighbors_bessel, where=num_neighbors_bessel != 0)
        )
    neighbor_std = np.sqrt(neighbor_var)
    # combine the results
    neighbor_data = np.column_stack([rag.nodes, num_neighbors, neighbor_mean, neighbor_std])
    neighbor_attrs_df = pd.DataFrame(neighbor_data).set_index(0)
    neighbor_feats = neighbor_attrs_df.fillna(0)
    neighbor_feats.index.names = ['label']
    n_bands_img_arr = img_arr.shape[-1] if len(img_arr.shape) == 3 else 1
    neighbor_feats.columns = [
        'neighbours_n',
        *[f'neighbours_mean_{i}' for i in np.arange(n_bands_img_arr)],
        *[f'neighbours_std_{i}' for i in np.arange(n_bands_img_arr)],
        ]
    return neighbor_feats.reset_index()

# perform calculation & get timing
neighb_vals = calc_mean_std_neighbours(segments, rgb)
display(neighb_vals)

Note that there are also some contextual features that doesn't require the calculation of a RAG in the first place. Among these descriptors, you can find the [Euler number](https://sydney4.medium.com/the-euler-number-of-a-binary-image-1d3cc9e57caa).

**<span style="color:orange">Task 9: Implement the calculation of the Euler number below. Evaluate its value in the given case.</span>**  

<i>Answer goes here</i>

In [ ]:
# define stdv across bands function
@timeit
def calc_euler(seg_arr):
    # code goes here

## Synergy & Use case

Putting everything together, you can now calculate a set of more than 20 features to describe the segments holistically. The expressiveness of these features shall be demonstrated below by using them as input for a random forest based classification of objects. In particular, we will try to create a binary classifier to identify the trees in the image. To this end, some samples in the original image are labelled and used together with some negative samples (representing the trees shadows, water, parking lots, etc) as input to a random forest. To evaluate the classifier, we will use a different subset spatially next to the one used for training, segment it analogously to the first one and apply the trained random forest to retrieve the predictions. 

In [ ]:
# calculate set of various features
@timeit
def calc_all_feats(seg_arr, rgbndvi_arr):
    ndvi_arr = rgbndvi_arr[...,-1]
    spec_feats = pd.DataFrame(
        regionprops_table(
            label_image = seg_arr, 
            intensity_image = rgbndvi_arr,
            properties = ["label", "intensity_mean"]
        )
    )
    shp_feats = pd.DataFrame(
        regionprops_table(
            label_image = seg_arr, 
            properties = ["moments_hu"],
        )
    )
    text_feats_I = pd.DataFrame(
        regionprops_table(
            label_image = seg_arr, 
            intensity_image = rgbndvi_arr,
            properties = [],
            extra_properties=(std,)
        )
    )
    text_feats_II = calc_entropy(seg_arr, ndvi_arr).iloc[:,1:]
    context_feats = calc_mean_std_neighbours(seg_arr, rgbndvi_arr).iloc[:,1:]
    all_feats = pd.concat([spec_feats, shp_feats, text_feats_I, text_feats_II, context_feats], axis=1)
    return all_feats  

# perform calculation & get timing
all_feats_df = calc_all_feats(segments, np.dstack([rgb,ndvi]))

In [ ]:
# samples for both classes
trees_idxs = [
    1204, 1186, 1134, 1229, 1329, 1201, 1140, 1218, 1351, 962, 830, 886, 926, 1373, 1405, 1517, 1547, 1461, 1526, 1406, 1358, 1460, 1556, 735, 833, 838, 802, 513, 616, 565, 438, 258, 375, 334, 284, 204, 250, 155, 183, 241, 344, 359, 412, 328, 424, 506, 468, 554, 459, 579, 560, 769, 890, 791, 798, 811, 771, 846, 1076, 784, 919, 915, 133, 246, 356, 405, 425, 474, 166, 158, 34, 200, 209, 245, 55, 2003, 1984, 1910, 1813, 1873, 1923, 2296, 2220, 2143, 2113, 2153, 2319, 2275, 2284, 2422, 2451, 2398, 2460, 2455, 2208, 2218, 2199, 2036, 1986, 1924
]

other_idxs = [
    2647, 2653, 2717, 2784, 2110, 2061, 2122, 1988, 1943, 2018, 1861, 1896, 1909, 1900, 1829, 1758, 1719, 1703, 1794, 1812, 1749, 1581, 1580, 1522, 1451, 1438, 1479, 1408, 1511, 1631, 1869, 1811, 1978, 2104, 2187, 2299, 2400, 2732, 2642, 2567, 2348, 2369, 2401, 2281, 1850, 1117, 197, 1002, 872, 216, 432, 418, 357, 382, 419, 336, 295, 351, 460, 644, 708, 800, 902, 1005, 1063, 174, 824, 831, 780, 1291, 1412, 1537, 1497, 1570, 1605, 1491, 1595, 1652, 1768, 1807, 1887, 1898, 1968, 1573, 1540, 1536, 1439, 1388, 1694, 2640 
]

# map sample_idxs to 2D grid
trees_bool = [2 if x in trees_idxs else 0 for x in np.array(means["label"])]
other_bool = [1 if x in other_idxs else 0 for x in np.array(means["label"])]
both_labels = [t if t else o for t,o in zip(trees_bool, other_bool)]

samples_df = pd.DataFrame({
    "id": np.array(means["label"]),
    "label": both_labels
})

mapped_samples = map_array(
    segments, 
    np.array(samples_df["id"]), 
    np.array(samples_df["label"])
    )

# visualise drawn samples
background_c, others_c, trees_c = '#7f8282', '#00008b', '#00FF00'
binary_cmap = mcolors.ListedColormap([background_c, others_c, trees_c])
plt.imshow(mark_boundaries(rgb, segments, (1,0,0), mode="outer"))
plt.imshow(mapped_samples, cmap=binary_cmap, alpha=0.5)
plt.axis("off");

In [ ]:
# extract feature data for all samples
tree_samples = all_feats_df[all_feats_df["label"].isin(trees_idxs)]
tree_samples.insert(0, "gt", 1)
other_samples = all_feats_df[all_feats_df["label"].isin(other_idxs)]
other_samples.insert(0, "gt", 0)
feats_samples = pd.concat([tree_samples, other_samples]).drop(columns="label")
feats_samples

In [ ]:
# instantiate random forest & grid search hyperparameter tuning
rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    oob_score=True
)
grid_search = GridSearchCV(
    rf,
    {"max_features": np.arange(0.05, 0.55, 0.05)},
    scoring="accuracy",
    cv=RepeatedKFold(n_splits=3, n_repeats=5, random_state=42),
    n_jobs=-1,
    verbose=1
)

# fit random forest & evaluate quantitatively
X = feats_samples.iloc[:,1:]
y = feats_samples.iloc[:,0]
grid_search.fit(X, y)
preds = grid_search.predict(all_feats_df.iloc[:,1:])

print(f"Out-of-bag accuarcy RF: {grid_search.best_estimator_.oob_score_:.2f}")

In [ ]:
# read & segment test image
img_path_test = "sample_data/ortho_subset_II.tif"
ds_test = xr.open_dataset(img_path_test)
arr_test = np.transpose(np.array(ds_test.to_array()[0]), (1,2,0))
rgb_test = arr_test.astype(np.uint8)[...,[0,1,2]]
ndvi_test = (arr_test[...,3] - arr_test[...,0])/(arr_test[...,3] + arr_test[...,0])
segments_test = slic(arr_test, n_segments=3000, compactness=0.3)

# calculate features for test set
feats_df_test = calc_all_feats(segments_test, np.dstack([rgb_test, ndvi_test]))

# get predictions for test set
preds_test = grid_search.predict(feats_df_test.iloc[:,1:])

In [ ]:
# visualise final results
mapped_preds = map_array(segments, np.unique(segments), preds)
mapped_preds_test = map_array(segments_test, np.unique(segments_test), preds_test)

fig, axs = plt.subplots(ncols=4, figsize=(20,10), constrained_layout=True)
axs[0].imshow(rgb)
axs[1].imshow(mapped_preds, cmap=binary_cmap, interpolation="nearest")
axs[2].imshow(rgb_test)
axs[3].imshow(mapped_preds_test, cmap=binary_cmap, interpolation="nearest")

axs[0].set_title("train image")
axs[1].set_title("trees prediction")
axs[2].set_title("test image")
axs[3].set_title("trees prediction")

for ax in axs.flat:
    ax.set_axis_off();

As you can see, the model's predictions are far from perfect but not too bad for a model that has been trained on just a handful of samples and features (less than 200 and 25, respectively). Note also that a pixel-wise classification would never achieve such accuracies and that deep learning approaches are more accurate but usually need hundreds or thousands of training images - our small model here runs on 1 sparsely annotated training image ;)        

## Conclusion & Further thoughts

This tutorial has showcased possibilities to calculate various feature descriptors. While it can be generalized that some features are commonly more expressive than others, the exact choice of the descriptors depends on the segmentation and the use case at hand. Other important considerations include the... 
* interpretability of features - some having a very intuitive meaning while others are less interpretable    
* scalability of calculations - esp. in the era of big data relevant as soon as we want to apply models to larger amounts of data

Further directions that may be interesting to explore are larger scale experiments on which feature sets appear to be most expressive for certain use cases. Training machine learning models such as random forests on different feature sets for different datasets and identifying the smallest, computationally most efficient feature set that allows to achieve high accuracies for a given task may be of broader interest especially in the context of the rise of deep learning models. While training deep learning models requires a lot of sample data and takes a relatively long time, the inference time (i.e. the time to apply the model to unseen samples) is usually very low. The development of a competitive non-deep learning based OBIA method therefore requires more than ever a focus on small, meaningful and efficiently computable feature sets in order to keep up.    